# EDA & Modélisation - Marché Électrique Danemark (DK1)
*Analyse des prix Day-Ahead et impact de l'énergie éolienne*

### 1. Configuration et Importation

In [27]:
import urllib.request
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
import shap
import warnings

warnings.filterwarnings('ignore')

# --- Téléchargement des données ---
os.makedirs('../data/raw', exist_ok=True)
url = "https://data.open-power-system-data.org/time_series/latest/time_series_60min_singleindex.csv"
destination = "../data/raw/time_series_60min.csv"

if not os.path.exists(destination):
    print("⏳ Téléchargement du dataset (ceci peut prendre un moment)...")
    urllib.request.urlretrieve(url, destination)
    print("Dataset téléchargé !")
else:
    print("Dataset déjà présent localement.")

# Chargement
df = pd.read_csv(destination, parse_dates=['utc_timestamp'], low_memory=False)
df = df.set_index('utc_timestamp')

Dataset déjà présent localement.


### 2. Préparation des Données (Zone DK1)

In [28]:
# Sélection des colonnes clés pour le Danemark Ouest (DK1) ou en core (Ouest - Jylland)
cols_mapping = {
    'DK_1_price_day_ahead': 'price',
    'DK_1_load_actual_entsoe_transparency': 'load_actual',
    'DK_1_load_forecast_entsoe_transparency': 'load_forecast',
    'DK_1_solar_generation_actual': 'solar_generation',
    'DK_1_wind_generation_actual': 'wind_generation' # Onshore + Offshore combinés
}

df_dk = df[list(cols_mapping.keys())].rename(columns=cols_mapping)

### Analyse de la Complétude des Données (Data Completeness)
*Avant de choisir une période, vérifions la disponibilité des données par année.*

In [29]:
# Calcul du nombre de données valides (non-NaN) par année pour chaque colonne
yearly_counts = df_dk.groupby(df_dk.index.year).count()

# Visualisation de la disponibilité
fig = px.bar(
    yearly_counts, 
    barmode='group',
    title="Qualité des Données : Nombre d'observations valides par an",
    labels={"index": "Année", "value": "Nombre d'heures valides", "variable": "Variable"},
    template="plotly_white",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Ajout d'une ligne de référence pour une année complète (365 * 24 = 8760 heures)
fig.add_hline(y=8760, line_dash="dash", line_color="red", 
              annotation_text="Année Complète (~8760h)", annotation_position="top right")

fig.update_layout(xaxis_title="Année", yaxis_title="Heures de données disponibles")
fig.show()

# Affichage des pourcentages de remplissage pour validation numérique
full_year_hours = 8760
completeness_pct = (yearly_counts / full_year_hours * 100).round(1)
print("Pourcentage de données disponibles par année (basé sur 8760h) :")
display(completeness_pct.style.background_gradient(cmap='RdYlGn', vmin=90, vmax=100))

Pourcentage de données disponibles par année (basé sur 8760h) :


,price,load_actual,load_forecast,solar_generation,wind_generation
utc_timestamp,,,,,
2014,0.000000,0.000000,0.000000,0.000000,0.000000
2015,100.000000,100.000000,100.000000,100.000000,100.000000
2016,100.300000,100.300000,100.300000,100.300000,100.300000
2017,100.000000,100.000000,100.000000,100.000000,100.000000
2018,100.000000,100.000000,100.000000,100.000000,100.000000
2019,100.000000,100.000000,100.000000,100.000000,100.000000
2020,75.000000,75.100000,75.100000,75.000000,75.100000


### 3. Filtrage et Feature Engineering

In [30]:
# Focus sur 2018-2019 suite à l'analyse de qualité
df_dk = df_dk.loc['2018-01-01':'2019-12-31']
df_dk = df_dk.interpolate(method='linear').dropna()

# Feature Engineering Temporel
df_dk['hour'] = df_dk.index.hour
df_dk['day_of_week'] = df_dk.index.dayofweek # 0=Lundi, 6=Dimanche
df_dk['weekday_name'] = df_dk.index.day_name()
df_dk['month'] = df_dk.index.month

print(f"Données prêtes : {df_dk.shape}")
display(df_dk.head())

Données prêtes : (17520, 9)


,price,load_actual,load_forecast,solar_generation,wind_generation,hour,day_of_week,weekday_name,month
utc_timestamp,,,,,,,,,
2018-01-01 00:00:00+00:00,26.43,1741.92,1747.8,0.0,1849.85,0,0,Monday,1
2018-01-01 01:00:00+00:00,26.10,1657.52,1659.4,0.0,1609.74,1,0,Monday,1
2018-01-01 02:00:00+00:00,24.70,1594.26,1599.5,0.0,1606.32,2,0,Monday,1
2018-01-01 03:00:00+00:00,24.74,1535.88,1560.5,0.0,1612.06,3,0,Monday,1
2018-01-01 04:00:00+00:00,18.01,1505.07,1534.0,0.0,1617.73,4,0,Monday,1


### 4. Visualisations "Signature" pour le Danemark (Insights)

In [31]:
# 4.1. L'Effet du Vent sur les Prix (Scatter Plot)
# Insight : Observez la corrélation inverse. Quand le vent dépasse 2000-3000 MW, les prix s'effondrent.
fig = px.scatter(
    df_dk.sample(5000), # Échantillon pour alléger le rendu
    x="wind_generation", 
    y="price", 
    color="load_actual",
    title="Corrélation : Production Éolienne vs Prix (DK1)",
    labels={"wind_generation": "Production Éolienne (MW)", "price": "Prix (€/MWh)", "load_actual": "Demande (MW)"},
    template="plotly_white",
    opacity=0.6,
    trendline="lowess" # Ligne de tendance locale
)
fig.update_layout(height=600)
fig.show()


In [32]:
# 4.2. La "Heatmap" Temporelle des Prix
# Insight : Permet de voir instantanément le profil journalier typique. Notez les pics du matin (8h-9h) et du soir (17h-19h) en semaine, moins marqués le weekend.
heatmap_data = df_dk.groupby(['weekday_name', 'hour'])['price'].mean().reset_index()

# Ordonner les jours de la semaine
days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
heatmap_data['weekday_name'] = pd.Categorical(heatmap_data['weekday_name'], categories=days_order, ordered=True)
heatmap_data = heatmap_data.sort_values(['weekday_name', 'hour'])

# Transformation en matrice pour Plotly
z_data = heatmap_data.pivot(index='weekday_name', columns='hour', values='price')

fig = px.imshow(
    z_data,
    labels=dict(x="Heure de la journée", y="Jour de la semaine", color="Prix Moyen (€)"),
    x=z_data.columns,
    y=z_data.index,
    title="Heatmap Hebdomadaire des Prix Moyens (DK1)",
    color_continuous_scale="RdYlGn_r", # Rouge = Cher, Vert = Pas cher
    aspect="auto"
)
fig.update_xaxes(dtick=2) # Etiquettes toutes les 2 heures
fig.show()



In [33]:
# 4.3. La Courbe de Durée des Prix (Price Duration Curve)
# Insight : Classique des marchés de l'énergie. Elle montre la volatilité. La queue à droite montre les heures à prix négatifs.
sorted_price = df_dk['price'].sort_values(ascending=False).reset_index(drop=True)
sorted_price.index = (sorted_price.index / len(sorted_price)) * 100 # En pourcentage du temps

fig = px.area(
    x=sorted_price.index, 
    y=sorted_price.values,
    title="Courbe de Durée des Prix (Price Duration Curve)",
    labels={"x": "% du temps (Année)", "y": "Prix (€/MWh)"},
    template="plotly_white"
)
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Prix Zéro")
fig.show()

In [34]:
# ### 4.4. Visualisations Avancées (Propositions Spécifiques)

# --- A. Matrice de Corrélation (Heatmap) ---
# Insight : Visualiser les relations positives (Rouge) et négatives (Bleu)
corr = df_dk[['price', 'load_actual', 'wind_generation', 'solar_generation']].corr()

fig = px.imshow(
    corr,
    text_auto=".2f",
    aspect="auto",
    title="Matrice de Corrélation : Qui influence le Prix ?",
    color_continuous_scale="RdBu_r", # Rouge = Positif, Bleu = Négatif
    origin='lower'
)
fig.show()

In [35]:
# --- B. Profil Horaire par Saison (Faceted Plot) ---
# Création de la colonne Saison
def get_season(month):
    if month in [12, 1, 2]: return 'Hiver'
    elif month in [3, 4, 5]: return 'Printemps'
    elif month in [6, 7, 8]: return 'Été'
    else: return 'Automne'

df_dk['season'] = df_dk['month'].apply(get_season)
# Ajout de l'année (nécessaire pour l'analyse mensuelle / groupby par year)
df_dk['year'] = df_dk.index.year

# Calcul de la moyenne par saison et par heure
season_profile = df_dk.groupby(['season', 'hour'])['price'].mean().reset_index()

fig = px.line(
    season_profile,
    x='hour',
    y='price',
    color='season',
    title="Profil Horaire des Prix selon la Saison",
    labels={'price': 'Prix Moyen (€/MWh)', 'hour': 'Heure'},
    template="plotly_white",
    category_orders={"season": ["Hiver", "Printemps", "Été", "Automne"]} # Ordre logique
)
fig.show()

# --- C. Analyse des Prix Négatifs (Fréquence Mensuelle) ---
# Insight : Quand les prix négatifs surviennent-ils le plus souvent ?
negative_prices = df_dk[df_dk['price'] < 0].groupby(['year', 'month']).size().reset_index(name='count')
# Création d'une date fictive pour l'axe X
negative_prices['date'] = pd.to_datetime(negative_prices[['year', 'month']].assign(DAY=1))

fig = px.bar(
    negative_prices,
    x='date',
    y='count',
    title="Fréquence des Prix Négatifs (Nombre d'heures par mois)",
    labels={'count': "Nombre d'heures < 0€", 'date': 'Date'},
    color='count',
    color_continuous_scale='Reds'
)
fig.show()

### 5. Modélisation Prédictive (LightGBM)

In [36]:
# ### 5. Modélisation Prédictive (LightGBM)

# Feature Engineering pour le modèle (Lags)
df_model = df_dk.copy()
df_model['price_lag_24h'] = df_model['price'].shift(24)
df_model['wind_forecast_lag_24h'] = df_model['wind_generation'].shift(24) # Approx forecast
df_model = df_model.dropna()

features = ['hour', 'day_of_week', 'month', 'price_lag_24h', 'load_forecast', 'wind_forecast_lag_24h', 'solar_generation']
target = 'price'

# Split Train/Test
split = '2019-06-01'
X_train = df_model.loc[df_model.index < split, features]
y_train = df_model.loc[df_model.index < split, target]
X_test = df_model.loc[df_model.index >= split, features]
y_test = df_model.loc[df_model.index >= split, target]

# Entraînement
model = lgb.LGBMRegressor(random_state=42, n_estimators=500)
model.fit(X_train, y_train)

# Prédiction
y_pred = model.predict(X_test)
print(f"RMSE Test : {np.sqrt(mean_squared_error(y_test, y_pred)):.2f} €/MWh")

# Visualisation Prédiction
fig = px.line(title="Prédiction vs Réalité (Échantillon sur 2 semaines)")
fig.add_scatter(x=y_test.index[:336], y=y_test.values[:336], name="Réel", line_color="black")
fig.add_scatter(x=y_test.index[:336], y=y_pred[:336], name="Prédit", line_color="orange", line_dash="dot")
fig.show()


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 12360, number of used features: 7
[LightGBM] [Info] Start training from score 43.085669
RMSE Test : 11.61 €/MWh


### 6. Explicabilité du Modèle (SHAP)

In [37]:
# Insight : Notez comment le 'Wind' (Vent) est souvent la feature la plus impactante après le Lag de prix.
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# SHAP Summary Plot (version Plotly simplifiée via Bar)
shap_sum = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame({'feature': features, 'importance': shap_sum}).sort_values('importance', ascending=True)

fig = px.bar(
    importance_df, 
    x='importance', 
    y='feature', 
    orientation='h',
    title="Importance Globale des Features (SHAP)",
    template="plotly_white"
)
fig.show()